## Fantasy Football 2025 Projected Points ##

#### Merged QB, Flex, DST, K, ADP and Handcuff Datasets

In [28]:
from IPython.display import HTML
HTML("""
<style>
/* Put bg on page root (works in Voila + Notebook) */
html, body {
  height: 100%;
  min-height: 100%;
  background:
    linear-gradient(to bottom, rgba(0,0,0,.35), rgba(0,0,0,.35)),
    url('/voila/files/assets/football_bg.jpg'),
    url('files/assets/football_bg.jpg') !important;
  background-position: center center !important;
  background-repeat: no-repeat !important;
  background-attachment: fixed !important;
  background-size: cover !important;
}

/* Make every major Voila/Jupyter wrapper transparent */
.voila-container,
#notebook, #notebook-container,
.jp-Notebook, .jp-NotebookPanel, .jp-NotebookPanel-notebook,
.jp-LabShell, .jp-MainAreaWidget, .jp-Cell, .jp-Cell-outputArea,
.container, .notebook-app, .lab-app {
  background: transparent !important;
}

/* Your cards stay readable */
.draft-app .card {
  background: rgba(255,255,255,0.92) !important;
  -webkit-backdrop-filter: blur(6px);
  backdrop-filter: blur(6px);
  color: #111;
}

/* Optional: default text color outside cards */
body { color: #eaeaea; }
</style>
""")


In [29]:
import pandas as pd
import numpy as np
import re, html
from typing import Optional

# === FILE PATHS ===
flex_file = r"C:\Users\WilliamGuy\Desktop\AMBOT\Fantasy Football Draft Predictions\CSV Files\FantasyPros_Projected_Points_FLX.csv"
qb_file   = r"C:\Users\WilliamGuy\Desktop\AMBOT\Fantasy Football Draft Predictions\CSV Files\FantasyPros_Projected_Points_QB.csv"
dst_file  = r"C:\Users\WilliamGuy\Desktop\AMBOT\Fantasy Football Draft Predictions\CSV Files\FantasyPros_Projected_Points_DST.csv"
k_file    = r"C:\Users\WilliamGuy\Desktop\AMBOT\Fantasy Football Draft Predictions\CSV Files\FantasyPros_Projected_Points_K.csv"

# Point this to your NEW real-time ADP CSV (or leave as old file; auto-detect works)
adp_file  = r"C:\Users\WilliamGuy\Desktop\AMBOT\Fantasy Football Draft Predictions\CSV Files\FantasyPros_2025_Real_Time_ADP_Rankings.csv"

handcuff_file = r"C:\Users\WilliamGuy\Desktop\AMBOT\Fantasy Football Draft Predictions\CSV Files\FantasyPros_Fantasy_Football_RB_Handcuffs.csv"

# --- knobs to shape synthetic points for K/DST (top rank gets 'top', last rank gets 'bottom') ---
KDST_POINTS = {
    'K':   {'top': 140.0, 'bottom': 95.0},
    'DST': {'top': 135.0, 'bottom': 90.0},
}

# ---------- helpers ----------
def _norm_name(s: str) -> str:
    """Normalize player names for matching across files."""
    s = str(s).strip()
    s = re.sub(r"[\u2019’']", "'", s)     # curly -> straight apostrophes
    s = re.sub(r"\.", "", s)              # drop dots (e.g., D.K.)
    s = re.sub(r"\s+Jr\.?$", " Jr", s, flags=re.I)
    s = re.sub(r"\s+", " ", s)
    return s

def _extract_player_from_team_bye(val: str) -> str:
    """From 'Ja'Marr Chase CIN (10)' -> 'Ja'Marr Chase' (keeps val if pattern not present)."""
    s = str(val).strip()
    m = re.match(r"^(.*?)\s+[A-Z]{2,4}\s*\(\d{1,2}\)\s*$", s)
    if m:
        return m.group(1).strip()
    s = re.sub(r"\s+[A-Z]{2,4}\s*(\(\d{1,2}\))?\s*$", "", s)
    return s.strip()

def _coerce_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Unify common header variants to ['Player','Position','Projected_Points','RK','ADP']."""
    cols = {c.lower().strip(): c for c in df.columns}
    # Player
    for key in ('player', 'player name', 'player_name', 'name'):
        if key in cols:
            df = df.rename(columns={cols[key]: 'Player'})
            break
    # Position
    for key in ('pos', 'position'):
        if key in cols:
            df = df.rename(columns={cols[key]: 'Position'})
            break
    # Points
    for key in ('fpts', 'projected_points', 'points'):
        if key in cols:
            df = df.rename(columns={cols[key]: 'Projected_Points'})
            break
    # Rank
    for key in ('rk', 'rank'):
        if key in cols:
            df = df.rename(columns={cols[key]: 'RK'})
            break
    # ADP
    if 'adp' in cols and 'ADP' not in df.columns:
        df = df.rename(columns={cols['adp']: 'ADP'})
    return df

def _synthesize_points_from_rank(df: pd.DataFrame, pos: str) -> pd.DataFrame:
    """For K/DST sheets that lack Projected_Points, build a monotonic score from rank."""
    if 'Projected_Points' in df.columns and df['Projected_Points'].notna().any():
        return df
    if 'RK' not in df.columns:
        base = np.mean(list(KDST_POINTS[pos].values()))
        df['Projected_Points'] = base
        return df
    rng = KDST_POINTS[pos]
    top, bottom = rng['top'], rng['bottom']
    rk = pd.to_numeric(df['RK'], errors='coerce')
    rk_min, rk_max = rk.min(), rk.max()
    if pd.isna(rk_min) or pd.isna(rk_max):
        df['Projected_Points'] = np.mean([top, bottom])
        return df
    if rk_max == rk_min:
        df['Projected_Points'] = np.full(len(df), np.mean([top, bottom]))
        return df
    norm = 1.0 - (rk - rk_min) / (rk_max - rk_min)
    df['Projected_Points'] = bottom + (top - bottom) * norm
    return df

# --- robust CSV loader for messy vendor exports (extra commas, quotes, BOM, etc.) ---
def _safe_read_csv(path: str) -> pd.DataFrame:
    """Try C engine; fall back to python; last resort repair to keep column count consistent."""
    try:
        return pd.read_csv(path, encoding='utf-8-sig')
    except Exception:
        pass
    try:
        return pd.read_csv(
            path,
            engine='python',
            sep=',',
            quotechar='"',
            escapechar='\\',
            on_bad_lines='skip',
            encoding='utf-8-sig'
        )
    except Exception:
        pass
    import csv
    rows = []
    with open(path, 'r', encoding='utf-8-sig', errors='ignore', newline='') as f:
        reader = csv.reader(f)
        header = next(reader)
        base_len = len(header)
        for row in reader:
            if len(row) < base_len:
                row = row + [''] * (base_len - len(row))
            elif len(row) > base_len:
                row = row[:base_len-1] + [",".join(row[base_len-1:])]
            rows.append(row)
    return pd.DataFrame(rows, columns=header)

# --- Real-Time ADP parsing (rank + delta) ---
def _extract_realtime_fields(series: pd.Series) -> pd.DataFrame:
    """
    Accepts '5 -1', '7 +2', '23', '23.0', or even ' 11  +1 '.
    Returns DataFrame with ADP (float) and ADP_Delta (int).
    Guards against Excel serials (e.g., 45778) by setting ADP NaN (>500).
    """
    s = series.astype(str).str.replace("\u00A0", " ", regex=False).str.strip()
    m = s.str.extract(r'^\s*(?P<rank>\d+(?:\.\d+)?)\s*(?P<sign>[+\-])?\s*(?P<d>\d+)?')
    rank = pd.to_numeric(m['rank'], errors='coerce')
    # Excel serial dates show up as ~45000+; treat as missing so we can fallback
    rank = rank.mask(rank > 500, np.nan)

    sign = m['sign'].map({'+': 1, '-': -1}).fillna(0).astype(int)
    dval = pd.to_numeric(m['d'], errors='coerce').fillna(0).astype(int)
    delta = (sign * dval).astype(int)
    return pd.DataFrame({'ADP': rank, 'ADP_Delta': delta})

def _find_col_icontains(df: pd.DataFrame, *needles: str) -> Optional[str]:
    """Case-insensitive, space/nbsp-insensitive column finder; returns first match or None."""
    for c in df.columns:
        cl = str(c).lower().replace('\xa0', ' ')
        if any(n in cl for n in needles):
            return c
    return None

def _parse_adp_any_schema(df: pd.DataFrame) -> pd.DataFrame:
    """
    Accepts either:
      • Real-time ADP sheet (has 'PLAYER Team (Bye)', 'POS', 'Real-Time (?)', etc.)
      • Old FantasyPros ADP (already has 'Player' + 'ADP'/'RK')
    Returns ['Player','ADP','ADP_Delta'].
    """
    lower_cols = {c.lower(): c for c in df.columns}

    # Real-time detection via presence of "player team (bye)"
    if 'player team (bye)' in lower_cols or 'player\u00a0team (bye)' in lower_cols:
        name_col = lower_cols.get('player team (bye)', lower_cols.get('player\u00a0team (bye)'))
        df = df.rename(columns={name_col: 'PLAYER Team (Bye)'})

        # Preferred source: any column that looks like "real-time"
        real_col = _find_col_icontains(df, 'real-time', 'real time')
        avg_col  = _find_col_icontains(df, 'avg')
        rank_col = _find_col_icontains(df, 'rank')  # plain rank column

        out = pd.DataFrame()
        out['Player'] = df['PLAYER Team (Bye)'].map(_extract_player_from_team_bye).map(_norm_name)

        if real_col is not None:
            parsed = _extract_realtime_fields(df[real_col])
            out['ADP'] = parsed['ADP']
            out['ADP_Delta'] = parsed['ADP_Delta']
        else:
            out['ADP'] = np.nan
            out['ADP_Delta'] = 0

        # Fallback if Real-Time ADP was NaN (e.g., Excel serial): use RANK, else AVG
        if rank_col is not None:
            rk = pd.to_numeric(df[rank_col], errors='coerce')
            out['ADP'] = out['ADP'].fillna(rk)
        if avg_col is not None:
            avg = pd.to_numeric(df[avg_col], errors='coerce')
            out['ADP'] = out['ADP'].fillna(avg)

        return out

    # Old schema: normalize and keep Player/ADP (or fall back to RK)
    df = _coerce_cols(df)
    if 'Player' not in df.columns and 'Player Team (Bye)' in df.columns:
        df['Player'] = df['Player Team (Bye)'].map(_extract_player_from_team_bye)

    out = pd.DataFrame()
    out['Player'] = df['Player'].map(_norm_name)
    if 'ADP' in df.columns:
        out['ADP'] = pd.to_numeric(df['ADP'], errors='coerce')
    elif 'RK' in df.columns:
        out['ADP'] = pd.to_numeric(df['RK'], errors='coerce')
    else:
        out['ADP'] = np.nan
    out['ADP_Delta'] = 0
    return out

# --- UI helpers for green/red delta badge next to names ---
def _delta_badge_html(delta) -> str:
    """Return a small colored +/- badge or empty string."""
    if delta is None or pd.isna(delta) or int(delta) == 0:
        return ""
    delta = int(delta)
    color = "#16a34a" if delta > 0 else "#dc2626"   # green-600 / red-600
    sign  = "+" if delta > 0 else "−"               # U+2212 minus
    return f"<span style='margin-left:6px; font-weight:700; color:{color}'>{sign}{abs(delta)}</span>"

def _name_with_delta_html(name, delta) -> str:
    return f"{html.escape(str(name))}{_delta_badge_html(delta)}"

# === LOAD RAW (use robust reader for ALL vendor files) ===
df_flex    = _safe_read_csv(flex_file)
df_qb      = _safe_read_csv(qb_file)
df_dst     = _safe_read_csv(dst_file)
df_k       = _safe_read_csv(k_file)
df_adp_raw = _safe_read_csv(adp_file)
df_hc      = _safe_read_csv(handcuff_file)  # handcuffs

# === UNIFY HEADERS (projections) ===
df_flex = _coerce_cols(df_flex)
df_qb   = _coerce_cols(df_qb)
df_dst  = _coerce_cols(df_dst)
df_k    = _coerce_cols(df_k)

# Handcuff columns -> Starter/Handcuff (support a few variants)
hc_renames = {}
for cand in ['Projected Starter', 'Projected starter', 'Starter', 'starter', 'RB (Starter)']:
    if cand in df_hc.columns:
        hc_renames[cand] = 'Starter'
        break
for cand in ['Handcuff', 'handcuff', 'RB (Handcuff)']:
    if cand in df_hc.columns:
        hc_renames[cand] = 'Handcuff'
        break
if hc_renames:
    df_hc = df_hc.rename(columns=hc_renames)

# === FLEX/QB points: rename if needed and clean ===
for df in (df_flex, df_qb):
    if 'Position' not in df.columns and 'POS' in df.columns:
        df.rename(columns={'POS':'Position'}, inplace=True)
    if 'Projected_Points' not in df.columns and 'FPTS' in df.columns:
        df.rename(columns={'FPTS':'Projected_Points'}, inplace=True)

# FLEX positions like WR1 → WR, RB2 → RB, TE3 → TE  (expand=False prevents DF shape issues)
if 'Position' in df_flex.columns:
    df_flex['Position'] = df_flex['Position'].astype(str).str.extract(r'([A-Z]+)', expand=False)

# Normalize positions we know
df_qb['Position']  = 'QB'
df_dst['Position'] = 'DST'
df_k['Position']   = 'K'

# Synthesize points for K/DST if absent
df_dst = _synthesize_points_from_rank(df_dst, 'DST')
df_k   = _synthesize_points_from_rank(df_k,   'K')

# Keep only the columns we need
def _keep(df: pd.DataFrame) -> pd.DataFrame:
    keep = [c for c in ('Player','Position','Projected_Points') if c in df.columns]
    out = df[keep].copy()
    # ensure types
    if 'Projected_Points' in out.columns:
        out['Projected_Points'] = pd.to_numeric(out['Projected_Points'], errors='coerce')
    if 'Player' in out.columns:
        out['Player'] = out['Player'].astype(str).str.strip()
    return out

df_flex = _keep(df_flex)
df_qb   = _keep(df_qb)
df_dst  = _keep(df_dst)
df_k    = _keep(df_k)

# === MERGE ALL POSITIONS ===
players = pd.concat([df_flex, df_qb, df_dst, df_k], ignore_index=True)
players = players.dropna(subset=['Projected_Points'])

# ---------- Name-normalized merge with ADP (with RealTime delta; resilient) ----------
players['Player_norm'] = players['Player'].map(_norm_name)

# 1) Parse ADP table (normalized)
df_adp = _parse_adp_any_schema(df_adp_raw).dropna(subset=['Player']).copy()
df_adp['Player'] = df_adp['Player'].map(_norm_name)

# Keep Real-Time ADP if present; fallback only if ADP is completely missing
if 'ADP' not in df_adp.columns and 'AVG' in df_adp.columns:
    df_adp['ADP'] = pd.to_numeric(df_adp['AVG'], errors='coerce')
df_adp['ADP'] = pd.to_numeric(df_adp['ADP'], errors='coerce')


# 2) Build/ensure ADP_Delta from the raw ADP file's Real-Time column (robust)
def _parse_rt_delta(s: str) -> int:
    s = (str(s) if s is not None else "").strip()
    if not s or s.lower() == "nan":
        return 0
    # normalize dashes & map arrows to signs
    s = (s.replace('−', '-').replace('—', '-')
         .replace('▲','+').replace('△','+').replace('↑','+').replace('⬆','+')
         .replace('▼','-').replace('▽','-').replace('↓','-').replace('⬇','-'))
    # require explicit sign; tolerate spaces/parentheses: "4  +1", "( -2 )", "↓ 3"
    m = re.search(r'([+\-]\s*\d+)', s)
    return int(m.group(1).replace(' ', '')) if m else 0

def _find_rt_col(df: pd.DataFrame) -> Optional[str]:
    for c in df.columns:
        cl = str(c).lower().replace('\xa0', ' ')
        if ('real' in cl) and ('time' in cl):
            return c
    return None

# find the correct name column in RAW for RT merge
def _find_raw_name_col(df: pd.DataFrame) -> Optional[str]:
    # Prefer the explicit "PLAYER Team (Bye)"-like column
    for c in df.columns:
        cl = str(c).lower().replace('\xa0',' ')
        if 'player' in cl and 'team' in cl and '(bye' in cl:
            return c
    # fall back to any column that includes 'player'
    for c in df.columns:
        if 'player' in str(c).lower():
            return c
    return None

rt_col   = _find_rt_col(df_adp_raw)
name_raw = _find_raw_name_col(df_adp_raw)

# Start with zeros; overwrite if we can parse Real-Time
df_adp['ADP_Delta'] = 0

if rt_col is not None and name_raw is not None:
    df_rt = df_adp_raw[[name_raw, rt_col]].copy()
    df_rt.rename(columns={name_raw: 'Player_raw'}, inplace=True)
    # Extract the pure name then normalize
    df_rt['Player'] = df_rt['Player_raw'].map(_extract_player_from_team_bye).map(_norm_name)
    df_rt['ADP_Delta'] = df_rt[rt_col].astype(str).apply(_parse_rt_delta).astype(int)
    # Deduplicate: keep strongest non-zero signal first
    df_rt = (df_rt.sort_values(by='ADP_Delta', ascending=False)
                  .drop_duplicates(subset=['Player'], keep='first'))
    # Left join onto the parsed/normalized ADP table
    df_adp = df_adp.merge(df_rt[['Player','ADP_Delta']],
                          on='Player', how='left', suffixes=('', '_rt'))
    df_adp['ADP_Delta'] = (df_adp['ADP_Delta_rt']
                           .fillna(df_adp['ADP_Delta'])
                           .astype('Int64')
                           .fillna(0)
                           .astype(int))
    if 'ADP_Delta_rt' in df_adp.columns:
        df_adp.drop(columns=['ADP_Delta_rt'], inplace=True)

# Final coercion (safety)
df_adp['ADP_Delta'] = pd.to_numeric(df_adp['ADP_Delta'], errors='coerce').fillna(0).astype(int)

# 3) Join onto players
players = players.merge(
    df_adp.rename(columns={'Player':'Player_norm'})[['Player_norm','ADP','ADP_Delta']],
    on='Player_norm', how='left'
)

# Keep canonical names; drop temp column
PLAYER_NAME_INDEX = dict(zip(players['Player_norm'], players['Player']))
players.drop(columns=['Player_norm'], inplace=True)

# Rank / sort
players = players.sort_values(by=['ADP','Projected_Points'], ascending=[True, False], kind='mergesort').reset_index(drop=True)
players.insert(0, 'Rank', players.index + 1)

# Badges for UI
players['ADP_Delta'] = pd.to_numeric(players.get('ADP_Delta', 0), errors='coerce').fillna(0).astype(int)
players['NameHTML'] = [
    _name_with_delta_html(n, int(d)) for n, d in zip(players['Player'], players['ADP_Delta'])
]

# ---------- Handcuff mappings (normalized) ----------
if all(c in df_hc.columns for c in ('Starter','Handcuff')):
    df_hc['Starter']  = df_hc['Starter'].map(_norm_name)
    df_hc['Handcuff'] = df_hc['Handcuff'].map(_norm_name)
    df_hc = df_hc.dropna(subset=['Starter','Handcuff'])
    STARTER_TO_CUFF = dict(zip(df_hc['Starter'], df_hc['Handcuff']))
    CUFF_SET = set(df_hc['Handcuff'])
    CUFF_TO_STARTER = {v: k for k, v in STARTER_TO_CUFF.items()}
else:
    # Safe empty defaults if the handcuff file is missing columns
    STARTER_TO_CUFF = {}
    CUFF_SET = set()
    CUFF_TO_STARTER = {}

# players DataFrame is ready; handcuff globals are available for the recommender cell
# Example: print(players[['Rank','Player','ADP','ADP_Delta']].head(25).to_string(index=False))


In [30]:
# === Build handcuff maps from your CSV (run this BEFORE the assistant cell) ===
import pandas as pd, re

# <-- change this path to your file -->
HANDCUFF_CSV = r"C:\Users\WilliamGuy\Desktop\AMBOT\Fantasy Football Draft Predictions\CSV Files\FantasyPros_Fantasy_Football_RB_Handcuffs.csv"

def _norm_name(s: str) -> str:
    s = str(s).strip()
    s = re.sub(r"[\u2019’']", "'", s)
    s = re.sub(r"\.", "", s)
    s = re.sub(r"\s+Jr\.?$", " Jr", s, flags=re.I)
    s = re.sub(r"\s+", " ", s)
    return s

df = pd.read_csv(HANDCUFF_CSV)

# Tolerate header variants
cols = {c.lower().strip(): c for c in df.columns}
starter_col  = cols.get("projected starter") or cols.get("starter") or "Projected Starter"
handcuff_col = cols.get("handcuff") or "Handcuff"
adp_col      = cols.get("adp") or "ADP"

# Keep only valid rows
df = df[[starter_col, handcuff_col, adp_col]].dropna(how="any")
df[starter_col]  = df[starter_col].astype(str).str.strip()
df[handcuff_col] = df[handcuff_col].astype(str).str.strip()
df[adp_col]      = df[adp_col].astype(str).str.strip()

# Helper: pull a round number from text like "16th round" → 16
def _extract_round_num(adp_text: str):
    m = re.search(r"(\d+)", str(adp_text))
    return int(m.group(1)) if m else None

# Build name index for nice display
PLAYER_NAME_INDEX = {}

# Map: handcuff → starter (normalized)
CUFF_TO_STARTER = {}
for _, row in df.iterrows():
    starter_disp  = row[starter_col]
    cuff_disp     = row[handcuff_col]
    adp_text      = row[adp_col]  # already human-friendly (e.g., "16th round")

    s_norm = _norm_name(starter_disp)
    c_norm = _norm_name(cuff_disp)

    PLAYER_NAME_INDEX.setdefault(s_norm, starter_disp)
    PLAYER_NAME_INDEX.setdefault(c_norm, cuff_disp)

    CUFF_TO_STARTER[c_norm] = s_norm

# Optional: starter → (handcuff + ADP text/num) for the reminder card
HANDCUFF_META = {}
for _, row in df.iterrows():
    starter_disp  = row[starter_col]
    cuff_disp     = row[handcuff_col]
    adp_text      = row[adp_col]
    adp_round     = _extract_round_num(adp_text)  # integer round if present

    s_norm = _norm_name(starter_disp)
    HANDCUFF_META[s_norm] = {
        "handcuff": cuff_disp,
        "adp_text": adp_text,             # e.g., "16th round"
        "adp_num": float("nan") if adp_round is None else adp_round,  # used only as fallback
    }

#print(f"Handcuff table loaded: {len(CUFF_TO_STARTER)} links")
# Example sanity check (adjust names if desired):
# print('Example ->', HANDCUFF_META.get(_norm_name('Bijan Robinson')))


In [ ]:
# ===========================
# Live Draft Assistant v4.5 (with Real-Time ADP delta badges) — ADP_RT DISPLAY ENABLED
# ===========================
import pandas as pd
from ipywidgets import (
    VBox, HBox, Button, HTML, IntText, Text, Output, Layout,
    ToggleButtons, IntSlider
)
from IPython.display import display, clear_output
from typing import List, Dict, Any, Optional
import re, math, html

# ---------------------------
# CONFIG
# ---------------------------
ROUNDS = 16
TEAMS = 10
YOUR_TEAM_DEFAULT = 3

# Starter plan: SOFT preference (R1–R8), not a hard constraint
STARTER_ROUNDS = 8
STARTER_TARGETS = {'QB': 1, 'RB': 3, 'WR': 3, 'TE': 1}

# Do NOT suggest K/DST before this round
KDST_EARLIEST_ROUND = 13

# Hard caps so 16 players fit comfortably
POSITIONAL_LIMITS = {'QB': 1, 'RB': 7, 'WR': 8, 'TE': 1, 'DST': 1, 'K': 1}
HARD_CAP_SET = {'QB', 'TE', 'K', 'DST'}  # NEVER suggest more than one of these

# Soft minimums (for feasibility only — never block above these)
MIN_REQUIREMENTS = {'RB': 2, 'WR': 2}

# Replacement baselines
REPLACEMENT_POINTS = {'QB': 270.0, 'RB': 190.0, 'WR': 180.0, 'TE': 160.0, 'DST': 120.0, 'K': 115.0}

# ADP/EV knobs
SUGGEST_TOP_K = 4
CANDIDATES_PER_POS = 8

# Global ADP strength (you already added something like this)
ADP_WEIGHT_SCALE = 0.50  # was 0.35; raise a bit so ADP still matters

# Soft "don’t reach too far" guard
REACH_GUARD_PICKS = TEAMS          # ~one round early is OK to consider
REACH_PENALTY_PER_PICK = 0.14      # penalty per extra pick beyond the guard
REACH_ALLOW_VALUE_MARGIN = 12.0    # waive penalty if player is this many pts better than best alt


# ADP sensitivity
ADP_DEADZONE = 8        # broadened so points dominate more often when ADP is "close"
ADP_SIGMA = 8.0

# Soft balance (RB/WR) in starter rounds
BALANCE_BONUS_SCALE = 0.35 # per missing unit vs. STARTER_TARGETS (RB/WR only)

# --- Starter completion emphasis ---
STARTER_COMPLETION_BONUS = 1.25     # strong push to finish unmet starters in R1–R8
OVERFILLED_POSITION_PENALTY = -1.0 # nudge away from positions already met when others unmet

# --- Value-steal override (new) ---
VALUE_STEAL_MARGIN = 10.0   # pts over best unmet-starter option to count as a steal
VALUE_STEAL_BONUS  = 0.90   # bonus when it's a steal

# --- HARD early RB cap for YOU ---
EARLY_RB_MAX = 3  # <= Round 8

# Handcuff priority
HANDCUFF_ADP_BONUS = 15
HANDCUFF_START_ROUND = STARTER_ROUNDS + 1  # begin boosting after starters (Round 9)

# THREAT model knobs
THREAT_BOOST = 0.9
THREAT_BENCH_RBWR_WEIGHT = 1.0
THREAT_BENCH_QBTE_NO_STARTER = 0.65
THREAT_BENCH_QBTE_HAS_STARTER = 0.05
THREAT_STARTER_WEIGHT = 1.0

# Suppress early QB/TE urgency for OTHER teams through Round 4 (forecast)
EARLY_QBTE_SUPPRESS_ROUND = 4
EARLY_QBTE_NEED_MULTIPLIER = 0.35

# ---------------------------
# Normalizers / handcuff plumbing
# ---------------------------

def _norm_name(s: str) -> str:
    s = str(s).strip()
    s = re.sub(r"[\u2019’']", "'", s)
    s = re.sub(r"\.", "", s)
    s = re.sub(r"\s+Jr\.?$", " Jr", s, flags=re.I)
    s = re.sub(r"\s+", " ", s)
    return s

# Globals supplied by your prep cell; provide safe fallbacks
CUFF_TO_STARTER: Dict[str,str] = globals().get("CUFF_TO_STARTER", {})
PLAYER_NAME_INDEX: Dict[str,str] = globals().get("PLAYER_NAME_INDEX", {})
HANDCUFF_META: Dict[str,Dict[str,Any]] = globals().get("HANDCUFF_META", {})
STARTER_TO_CUFF: Dict[str,str] = globals().get("STARTER_TO_CUFF", {})

def _adp_num_to_round_text(adp_num: float) -> str:
    if pd.isna(adp_num): return "—"
    r = int((float(adp_num) - 1) // TEAMS) + 1
    return f"Round {r}"

def handcuff_card_html(starter_name: str) -> str:
    s_norm = _norm_name(starter_name)
    meta = HANDCUFF_META.get(s_norm)
    if not meta: return ""
    cuff_name = meta.get('handcuff', '')
    adp_text  = meta.get('adp_text') or _adp_num_to_round_text(meta.get('adp_num', float('nan')))
    return f"""
<div class="hc-card" style="border:1px solid #ddd;padding:8px;border-radius:8px;background:#fff8e1;max-width:460px;margin:6px;">
  <div style="font-weight:600;margin-bottom:6px;">Handcuff reminder</div>
  <div><b>{cuff_name}</b> — average draft: {adp_text}</div>
  <div style="font-size:12px;color:#666;margin-top:4px;">
    Consider stashing later as insurance for {starter_name}.
  </div>
</div>"""

# ===========================
# Handcuff meta auto-builder (in case your prep cell didn't set HANDCUFF_META)
# ===========================
if not HANDCUFF_META and STARTER_TO_CUFF and 'players' in globals():
    # Build cuff ADP lookup from players
    p_norm = players[['Player','ADP']].copy()
    p_norm['norm'] = p_norm['Player'].map(_norm_name)
    cuff_adp = dict(zip(p_norm['norm'], p_norm['ADP']))
    for starter, cuff in STARTER_TO_CUFF.items():
        s_norm = _norm_name(starter)
        c_norm = _norm_name(cuff)
        adp_num = cuff_adp.get(c_norm, float('nan'))
        HANDCUFF_META[s_norm] = {
            'handcuff': cuff,
            'adp_num': adp_num,
            'adp_text': _adp_num_to_round_text(adp_num)
        }
    # Ensure CUFF_TO_STARTER exists
    if not CUFF_TO_STARTER:
        CUFF_TO_STARTER = { _norm_name(v): _norm_name(k) for k, v in STARTER_TO_CUFF.items() }

# ===========================
# Persistent handcuff state
# ===========================
handcuff_reminders: Dict[str, Dict[str, Any]] = {}

def render_handcuff_panel():
    if not handcuff_reminders:
        handcuff_box.value = ""
        return
    cards_html = []
    for s_norm, meta in handcuff_reminders.items():
        cuff_name = meta.get("handcuff", "")
        adp_text  = meta.get("adp_text") or _adp_num_to_round_text(meta.get('adp_num', float('nan')))
        starter_display = meta.get("starter_display", "")
        cards_html.append(f"""
<div class="hc-card" style="border:1px solid #ddd;padding:8px;border-radius:8px;background:#fff8e1;margin:6px;min-width:220px;">
  <div style="font-weight:600;margin-bottom:6px;">Handcuff reminder</div>
  <div><b>{cuff_name}</b> — average draft: {adp_text}</div>
  <div style="font-size:12px;color:#666;margin-top:4px;">
    Insurance for {starter_display}.
  </div>
</div>""")
    handcuff_box.value = f"<div id='handcuff-reminders' style='display:flex;flex-wrap:wrap;align-items:flex-start;'>{''.join(cards_html)}</div>"

def rebuild_handcuff_state_from_roster():
    handcuff_reminders.clear()
    drafted_set = {_norm_name(p) for p in drafted_players}
    for slot in your_roster:
        if slot['Position'] != 'RB':
            continue
        starter_norm = _norm_name(slot['Player'])
        meta = HANDCUFF_META.get(starter_norm)
        if not meta:
            continue
        cuff_norm = _norm_name(meta.get('handcuff', ''))
        if cuff_norm in drafted_set:
            continue
        handcuff_reminders[starter_norm] = {"starter_display": slot.get('Player'), **meta}
    render_handcuff_panel()

def add_handcuff_reminder_if_applicable(starter_name: str):
    s_norm = _norm_name(starter_name)
    if s_norm not in HANDCUFF_META: return
    handcuff_reminders[s_norm] = {"starter_display": starter_name, **HANDCUFF_META[s_norm]}
    render_handcuff_panel()

def remove_handcuff_reminder_if_cuff(picked_name: str):
    p_norm = _norm_name(picked_name)
    starter_norm = CUFF_TO_STARTER.get(p_norm)
    if starter_norm and starter_norm in handcuff_reminders:
        del handcuff_reminders[starter_norm]
        render_handcuff_panel()

# ---------------------------
# EXPECTED INPUT
# ---------------------------
if 'players' not in globals():
    raise RuntimeError(
        "Please create a DataFrame named `players` with columns: "
        "['Player','Position','Projected_Points','ADP'] (ADP numeric)."
    )

players = players.copy()
players['Player'] = players['Player'].astype(str).str.strip()

# --- Position normalization ---
POS_ALIASES = {
    'DEF': 'DST', 'D/ST': 'DST', 'TEAM DEFENSE': 'DST', 'DST': 'DST',
    'PK': 'K', 'K': 'K', 'QB': 'QB', 'TE': 'TE', 'RB': 'RB', 'WR': 'WR'
}
players['Position'] = (
    players['Position'].astype(str).str.upper().str.strip()
      .map(POS_ALIASES).fillna(players['Position'].astype(str).str.upper().str.strip())
)

players['Projected_Points'] = pd.to_numeric(players['Projected_Points'], errors='coerce')

# ---------------------------
# ADP / NameHTML hookup (rely on prep step; don't re-parse Real-Time here)
# ---------------------------

# Ensure ADP column is numeric (fallback to AVG if present)
if 'ADP' not in players.columns and 'AVG' in players.columns:
    players['ADP'] = pd.to_numeric(players['AVG'], errors='coerce')
players['ADP'] = pd.to_numeric(players.get('ADP', float('nan')), errors='coerce')

# Ensure ADP_Delta exists (prep step normally provides it)
if 'ADP_Delta' not in players.columns:
    players['ADP_Delta'] = 0
players['ADP_Delta'] = pd.to_numeric(players['ADP_Delta'], errors='coerce').fillna(0).astype(int)

def _delta_badge_html(d: int) -> str:
    try:
        di = int(d)
    except:
        return ""
    if di == 0:
        return ""
    color = "#16a34a" if di > 0 else "#dc2626"   # green up, red down
    sign  = "+" if di > 0 else "−"
    return f"<span style='margin-left:6px;font-weight:700;color:{color}'>{sign}{abs(di)}</span>"

# (Re)build NameHTML so badges appear in grid + roster (idempotent)
players['NameHTML'] = [
    f"{html.escape(str(n))}{_delta_badge_html(d)}"
    for n, d in zip(players['Player'], players['ADP_Delta'])
]

# --- Real-time ADP for DISPLAY ONLY (keep raw ADP for logic) ---
_adp_raw   = pd.to_numeric(players['ADP'], errors='coerce')
_adp_delta = pd.to_numeric(players['ADP_Delta'], errors='coerce')
players['ADP_RT'] = (_adp_raw - _adp_delta).round()
players['ADP_RT'] = players['ADP_RT'].where(players['ADP_RT'].notna(), _adp_raw)

# ---------------------------
# DRAFT ORDER (snake)
# ---------------------------

def get_full_draft_order(teams: int = TEAMS, rounds: int = ROUNDS):
    return [
        team
        for rnd in range(rounds)
        for team in (range(1, teams + 1) if rnd % 2 == 0 else range(teams, 0, -1))
    ]

draft_order = get_full_draft_order(TEAMS, ROUNDS)

# ---------------------------
# STATE
# ---------------------------

drafted_players: List[str] = []
your_counts: Dict[str, int] = {k: 0 for k in POSITIONAL_LIMITS}
your_roster: List[Dict[str, Any]] = []
pick_index = 0

# Track all team rosters for threat modeling
team_counts: Dict[int, Dict[str, int]] = {t: {k: 0 for k in POSITIONAL_LIMITS} for t in range(1, TEAMS+1)}

# History (for Undo)
pick_history: List[Dict[str, Any]] = []

# ---------------------------
# HELPERS
# ---------------------------

def points_of(pick) -> float:
    return float(pick['Projected_Points'])

def remaining_your_picks(start_idx: int, your_team: int) -> int:
    return sum(1 for t in draft_order[start_idx:] if t == your_team)

def current_round(idx: int) -> int:
    return idx // TEAMS + 1

def on_the_clock() -> int:
    return draft_order[pick_index] if pick_index < len(draft_order) else -1

def dynamic_min_needed_soft(counts: Dict[str, int], idx: int) -> Dict[str, int]:
    return {pos: max(0, MIN_REQUIREMENTS.get(pos, 0) - counts.get(pos, 0)) for pos in MIN_REQUIREMENTS}

def allowed_positions_now(idx: int) -> set:
    rnd = current_round(idx)
    base = {'RB', 'WR'}
    if rnd >= KDST_EARLIEST_ROUND: base |= {'K', 'DST'}
    if rnd >= 3: base |= {'QB', 'TE'}
    return base

def qbte_gate_allows(idx: int, your_team: int) -> bool:
    return current_round(idx) >= 3

def allowed_positions_for_team(team: int, idx: int) -> set:
    rnd = current_round(idx)
    counts = team_counts[team]
    if rnd <= STARTER_ROUNDS:
        needed = {pos for pos, tgt in STARTER_TARGETS.items() if counts.get(pos, 0) < tgt}
        base = needed if needed else {'RB', 'WR'}
    elif rnd <= KDST_EARLIEST_ROUND - 1:
        base = {'RB', 'WR'}
        if counts.get('QB', 0) == 0:
            base |= {'QB'}
        if counts.get('TE', 0) == 0:
            base |= {'TE'}
    else:
        base = {'RB', 'WR', 'K', 'DST'}
        if counts.get('QB', 0) == 0:
            base.add('QB')
        if counts.get('TE', 0) == 0:
            base.add('TE')

    # Don’t allow hard-cap positions > 1
    base = {p for p in base if not (p in HARD_CAP_SET and counts.get(p, 0) >= 1)}
    if current_round(idx) < KDST_EARLIEST_ROUND:
        base -= {'K', 'DST'}
    return base


def feasible_after_pick(counts: Dict[str, int], pos: str, picks_left_for_you: int, idx: int) -> bool:
    if counts.get(pos, 0) + 1 > POSITIONAL_LIMITS.get(pos, 99): return False
    updated = counts.copy(); updated[pos] = updated.get(pos, 0) + 1
    need = dynamic_min_needed_soft(updated, idx)
    return sum(need.values()) <= max(0, picks_left_for_you - 1)

def best_available_player(available: pd.DataFrame, pos: str) -> Dict[str, Any]:
    pool = available[available['Position'] == pos]
    if not pool.empty:
        row = pool.iloc[0]
        return {'Player': str(row['Player']), 'Position': row['Position'], 'Projected_Points': float(row['Projected_Points'])}
    return {'Player': f"Replacement {pos}", 'Position': pos, 'Projected_Points': float(REPLACEMENT_POINTS.get(pos, 0))}

def add_pick_to_state(pick: Dict[str, Any], is_you: bool):
    global drafted_players, your_counts, your_roster, team_counts, pick_history
    team = draft_order[pick_index] if pick_index < len(draft_order) else None
    pos_norm = str(pick['Position']).upper().strip()
    pos_norm = POS_ALIASES.get(pos_norm, pos_norm)

    # Record history BEFORE mutation
    pick_history.append({'player': pick['Player'],'position': pos_norm,'team': team,'is_you': is_you})

    # Mutate state
    drafted_players.append(pick['Player'])
    if team in team_counts:
        team_counts[team][pos_norm] = team_counts[team].get(pos_norm, 0) + 1
    if is_you:
        your_counts[pos_norm] = your_counts.get(pos_norm, 0) + 1
        # Store NameHTML for roster UI
        try:
            name_html_val = players.loc[players['Player']==pick['Player'], 'NameHTML'].iloc[0]
        except Exception:
            name_html_val = html.escape(pick['Player'])
        your_roster.append({
            'Player': pick['Player'],
            'NameHTML': name_html_val,
            'Position': pos_norm,
            'Projected_Points': pick['Projected_Points']
        })

def overall_pick_number(idx: int) -> int:
    return idx + 1

def next_pick_index_for_you(start_idx: int, your_team: int) -> int:
    for j in range(start_idx, len(draft_order)):
        if draft_order[j] == your_team:
            return j
    return -1

def adp_survival_prob(adp: float, next_overall_pick: int, sigma: float = ADP_SIGMA) -> float:
    if pd.isna(adp): return 0.25
    z = (adp - next_overall_pick) / sigma
    return 1.0 / (1.0 + math.exp(-z))

# ---------------------------
# THREAT MODEL + availability
# ---------------------------

def current_available(pos_filter: str = "All", search_text: str = "") -> pd.DataFrame:
    av = players[~players['Player'].isin(drafted_players)]
    if pos_filter != "All":
        av = av[av['Position'] == pos_filter]
    if search_text:
        q = search_text.strip().lower()
        if q:
            av = av[av['Player'].str.lower().str.contains(q)]
    return av

def round_and_pick(idx: int) -> str:
    if idx >= len(draft_order): return "Draft complete"
    rnd = idx // TEAMS + 1; pk = idx % TEAMS + 1
    return f"Round {rnd}, Pick {pk}"

def intervening_pick_indices(your_team: int) -> list:
    if on_the_clock() != your_team: return []
    next_idx = next_pick_index_for_you(pick_index + 1, your_team)
    if next_idx == -1: return []
    return list(range(pick_index + 1, next_idx))

def position_threat_between_picks(pos: str, your_team: int) -> float:
    idxs = intervening_pick_indices(your_team)
    if not idxs: return 0.0
    pos = POS_ALIASES.get(pos.upper().strip(), pos.upper().strip())

    no_take_prob_product = 1.0
    for idx in idxs:
        team = draft_order[idx]
        allowed = allowed_positions_for_team(team, idx)
        if pos not in allowed: continue
        rnd = current_round(idx)
        counts = team_counts[team]

        if rnd <= STARTER_ROUNDS:
            def _w_need(p):
                n = max(0, STARTER_TARGETS.get(p, 0) - counts.get(p, 0))
                if rnd <= EARLY_QBTE_SUPPRESS_ROUND and p in {'QB','TE'}:
                    n *= EARLY_QBTE_NEED_MULTIPLIER
                return n
            need_left = _w_need(pos)
            total_need = sum(_w_need(p) for p in allowed)
            p_t = (need_left * THREAT_STARTER_WEIGHT) / total_need if total_need > 0 else 0.0
        else:
            if pos in {'RB', 'WR'}:
                denom = (('RB' in allowed) + ('WR' in allowed))
                p_t = THREAT_BENCH_RBWR_WEIGHT / max(1, denom)
            elif pos in {'QB','TE'}:
                p_t = THREAT_BENCH_QBTE_NO_STARTER if counts.get(p, 0) == 0 else THREAT_BENCH_QBTE_HAS_STARTER
            elif pos in {'K','DST'}:
                p_t = 0.35
            else:
                p_t = 0.1

        p_t = max(0.0, min(1.0, p_t))
        no_take_prob_product *= (1.0 - p_t)

    threat = 1.0 - no_take_prob_product
    return max(0.0, min(1.0, threat))

def per_team_pos_probs(team: int, idx: int) -> Dict[str, float]:
    allowed = allowed_positions_for_team(team, idx)
    if not allowed: return {}
    counts = team_counts[team]; rnd = current_round(idx); probs: Dict[str, float] = {}
    if rnd <= STARTER_ROUNDS:
        needs = {}
        for p in allowed:
            n = max(0, STARTER_TARGETS.get(p, 0) - counts.get(p, 0))
            if rnd <= EARLY_QBTE_SUPPRESS_ROUND and p in {'QB','TE'}:
                n *= EARLY_QBTE_NEED_MULTIPLIER
            needs[p] = n
        total_need = sum(needs.values())
        for p, n in needs.items(): probs[p] = (n * THREAT_STARTER_WEIGHT) if total_need > 0 else 0.0
    else:
        for p in allowed:
            if p in {'RB','WR'}:
                denom = (('RB' in allowed) + ('WR' in allowed))
                probs[p] = THREAT_BENCH_RBWR_WEIGHT / max(1, denom)
            elif p in {'QB','TE'}:
                probs[p] = THREAT_BENCH_QBTE_NO_STARTER if counts.get(p, 0) == 0 else THREAT_BENCH_QBTE_HAS_STARTER
            elif p in {'K','DST'}:
                probs[p] = 0.35
            else:
                probs[p] = 0.1
    s = sum(probs.values())
    if s > 0: probs = {k: v / s for k, v in probs.items()}
    return probs

# ---------------------------
# Handcuff-aware ADP adjuster
# ---------------------------

def _your_roster_norm_set() -> set:
    return {_norm_name(p['Player']) for p in your_roster}

def adjusted_adp_for_handcuff(player_name: str, raw_adp):
    if pd.isna(raw_adp): return raw_adp
    if current_round(pick_index) < HANDCUFF_START_ROUND: return raw_adp
    p_norm = _norm_name(player_name)
    starter_norm = CUFF_TO_STARTER.get(p_norm)
    if not starter_norm: return raw_adp
    if starter_norm in _your_roster_norm_set():
        return max(1.0, float(raw_adp) - HANDCUFF_ADP_BONUS)
    return raw_adp

# ---------------------------
# Starter completion term with value-steal override
# ---------------------------

def starter_completion_term(pos: str, pts_now: float, best_unmet_pts: float) -> float:
    if current_round(pick_index) > STARTER_ROUNDS: return 0.0
    unmet = {p for p, tgt in STARTER_TARGETS.items() if your_counts.get(p, 0) < tgt}
    if not unmet: return 0.0
    if pos in unmet: return STARTER_COMPLETION_BONUS
    if best_unmet_pts and pts_now >= best_unmet_pts + VALUE_STEAL_MARGIN:
        return VALUE_STEAL_BONUS
    return OVERFILLED_POSITION_PENALTY

# ---------------------------
# ADP-aware suggestions with THREAT blending + starter completion + soft balance
# ---------------------------

# ---------------------------
# ADP-aware suggestions with THREAT blending + starter completion + soft balance + reach guard
# ---------------------------

def suggest_top_k(k: int, your_team: int):
    if on_the_clock() != your_team: return []

    av = current_available(position_filter.value, search.value)
    picks_left = remaining_your_picks(pick_index, your_team)
    next_idx = next_pick_index_for_you(pick_index + 1, your_team)
    next_overall = overall_pick_number(next_idx) if next_idx != -1 else 10**9

    allowed_now = allowed_positions_now(pick_index)
    if not qbte_gate_allows(pick_index, your_team): allowed_now -= {'QB', 'TE'}
    if current_round(pick_index) <= STARTER_ROUNDS and your_counts.get('RB', 0) >= EARLY_RB_MAX:
        allowed_now -= {'RB'}

    candidates = []
    for pos in POSITIONAL_LIMITS.keys():
        if pos not in allowed_now: continue
        if your_counts.get(pos, 0) >= POSITIONAL_LIMITS[pos]: continue
        if pos in HARD_CAP_SET and your_counts.get(pos, 0) >= 1: continue
        if not feasible_after_pick(your_counts, pos, picks_left, pick_index): continue

        top_pos = av[av['Position'] == pos].nlargest(CANDIDATES_PER_POS, 'Projected_Points')
        for _, row in top_pos.iterrows():
            raw_adp = float(row['ADP']) if 'ADP' in row and pd.notna(row['ADP']) else float('nan')
            pname = str(row['Player'])
            adp_eff = adjusted_adp_for_handcuff(pname, raw_adp)
            candidates.append({
                'Player': pname,
                'NameHTML': row.get('NameHTML', pname),
                'ADP_Delta': int(row.get('ADP_Delta', 0)) if pd.notna(row.get('ADP_Delta', 0)) else 0,
                'Position': pos,
                'Projected_Points': float(row['Projected_Points']),
                'ADP': raw_adp,                         # raw for logic
                'ADP_eff': adp_eff,                     # handcuff-adjusted for logic
                'ADP_disp': row.get('ADP_RT', raw_adp)  # display ADP
            })

    if not candidates:
        for pos in POSITIONAL_LIMITS.keys():
            if pos not in allowed_now: continue
            if your_counts.get(pos, 0) >= POSITIONAL_LIMITS[pos]: continue
            if pos in HARD_CAP_SET and your_counts.get(pos, 0) >= 1: continue
            if not feasible_after_pick(your_counts, pos, picks_left, pick_index): continue
            top_pos = av[av['Position'] == pos].nlargest(CANDIDATES_PER_POS, 'Projected_Points')
            for _, row in top_pos.iterrows():
                raw_adp = float(row['ADP']) if 'ADP' in row and pd.notna(row['ADP']) else float('nan')
                pname = str(row['Player'])
                adp_eff = adjusted_adp_for_handcuff(pname, raw_adp)
                candidates.append({
                    'Player': pname,
                    'NameHTML': row.get('NameHTML', pname),
                    'ADP_Delta': int(row.get('ADP_Delta', 0)) if pd.notna(row.get('ADP_Delta', 0)) else 0,
                    'Position': pos,
                    'Projected_Points': float(row['Projected_Points']),
                    'ADP': raw_adp,
                    'ADP_eff': adp_eff,
                    'ADP_disp': row.get('ADP_RT', raw_adp)
                })

    threat_cache: Dict[str, float] = {}
    if next_idx != -1:
        for pos in set(c['Position'] for c in candidates):
            threat_cache[pos] = position_threat_between_picks(pos, your_team)
    else:
        for pos in set(c['Position'] for c in candidates):
            threat_cache[pos] = 0.0

    best_pts_by_pos: Dict[str, float] = {}
    for c in candidates:
        p = c['Position']
        best_pts_by_pos[p] = max(best_pts_by_pos.get(p, float('-inf')), c['Projected_Points'])

    def balance_bonus(pos: str) -> float:
        if current_round(pick_index) > STARTER_ROUNDS: return 0.0
        if pos not in {'RB','WR'}: return 0.0
        deficit = max(0, STARTER_TARGETS[pos] - your_counts.get(pos, 0))
        return BALANCE_BONUS_SCALE * deficit

    def adp_weight(adp_eff):
        if pd.isna(adp_eff):
            base = 0.5
        else:
            d = abs(adp_eff - next_overall)
            base = 0.0 if d <= ADP_DEADZONE else 1.0
        return base * ADP_WEIGHT_SCALE   # global attenuator you added earlier

    def backup_points_for_pos(pos: str) -> float:
        likely_pool = av[(av['Position'] == pos) & ((av['ADP'] >= next_overall) | (av['ADP'].isna()))]
        if likely_pool.empty: return float(REPLACEMENT_POINTS.get(pos, 0.0))
        return float(likely_pool['Projected_Points'].max())

    # --- Precompute best and second-best points for reach waiver logic ---
    pts_list = [c['Projected_Points'] for c in candidates] or [0.0]
    max_pts = max(pts_list)
    second_max_pts = max((v for v in pts_list if v < max_pts), default=max_pts)

    scored = []
    for cand in candidates:
        pts_now = cand['Projected_Points']
        adp_for_survival = cand['ADP_eff'] if pd.notna(cand['ADP_eff']) else cand['ADP']
        p_survive = adp_survival_prob(adp_for_survival, next_overall) if not pd.isna(adp_for_survival) else 0.25
        backup_pts = backup_points_for_pos(cand['Position'])

        w = adp_weight(adp_for_survival)
        ev_wait = w * (p_survive * pts_now + (1.0 - p_survive) * backup_pts) + (1 - w) * pts_now
        net_gain = pts_now - ev_wait

        tpos = threat_cache.get(cand['Position'], 0.0)
        threat_bonus = THREAT_BOOST * tpos

        unmet = set()
        if current_round(pick_index) <= STARTER_ROUNDS:
            unmet = {p for p, tgt in STARTER_TARGETS.items() if your_counts.get(p, 0) < tgt}
        best_unmet_pts = max((best_pts_by_pos[p] for p in unmet if p in best_pts_by_pos), default=0.0)

        completion_bonus = starter_completion_term(cand['Position'], pts_now, best_unmet_pts)
        soft_bonus = balance_bonus(cand['Position'])

        # --- Soft reach guard ---
        reach_penalty = 0.0
        adp_eff_for_reach = adp_for_survival
        if pd.notna(adp_eff_for_reach):
            # how many picks earlier than market would we be taking him (beyond the guard)?
            reach_picks = max(0.0, (adp_eff_for_reach - next_overall) - REACH_GUARD_PICKS)
            # compute best alternative (exclude this candidate if it's the top)
            best_other = second_max_pts if pts_now == max_pts else max_pts
            # waive penalty if this is a clear value smash vs other options
            if pts_now < (best_other + REACH_ALLOW_VALUE_MARGIN):
                reach_penalty = REACH_PENALTY_PER_PICK * reach_picks

        scored.append({
            **cand,
            'Score': (net_gain + threat_bonus + completion_bonus + soft_bonus - reach_penalty, pts_now),
            'Threat': tpos
        })

    scored.sort(key=lambda x: x['Score'], reverse=True)

    out = []
    for s in scored[:k]:
        out.append({
            'Player': s['Player'],
            'NameHTML': s.get('NameHTML', s['Player']),
            'ADP_Delta': s.get('ADP_Delta', 0),
            'Position': s['Position'],
            'Projected_Points': s['Projected_Points'],
            'ADP': s['ADP'],                 # raw (logic ref)
            'ADP_eff': s['ADP_eff'],         # logic
            'ADP_disp': s.get('ADP_disp'),   # display
            'Threat': s['Threat']
        })
    return out


def filter_suggestions_by_caps(recs, counts):
    out = []
    for r in recs:
        pos = str(r['Position']).upper().strip()
        pos = POS_ALIASES.get(pos, pos)
        if pos in HARD_CAP_SET and counts.get(pos, 0) >= 1: continue
        out.append({**r, 'Position': pos})
    return out

def apply_pick_by_name(pname: str, is_you: bool):
    global pick_index
    av = players[~players['Player'].isin(drafted_players)]
    row = av[av['Player'] == pname]
    if row.empty: return
    pos_raw = str(row.iloc[0]['Position'])
    pos = POS_ALIASES.get(pos_raw.upper().strip(), pos_raw.upper().strip())
    pick = {'Player': str(row.iloc[0]['Player']),
            'Position': pos,
            'Projected_Points': float(row.iloc[0]['Projected_Points'])}

    if is_you and pos in {'QB','TE'} and not qbte_gate_allows(pick_index, int(you_box.value)):
        suggest.value = f"<b>Not yet:</b> {pos} picks open starting Round 3."; return
    if is_you and pos in {'K','DST'} and current_round(pick_index) < KDST_EARLIEST_ROUND:
        suggest.value = f"<b>Not yet:</b> {pos} suggestions start in Round {KDST_EARLIEST_ROUND}."; return
    if is_you and pos == 'RB' and current_round(pick_index) <= STARTER_ROUNDS and your_counts.get('RB', 0) >= EARLY_RB_MAX:
        suggest.value = f"<b>RB cap reached:</b> Max {EARLY_RB_MAX} RB in first {STARTER_ROUNDS} rounds."; return

    if is_you and on_the_clock() == int(you_box.value):
        if pos in HARD_CAP_SET and your_counts.get(pos, 0) >= 1:
            suggest.value = f"<b>Cannot pick {pname}:</b> {pos} max (1) already reached."; return
        picks_left = remaining_your_picks(pick_index, int(you_box.value))
        if your_counts.get(pos, 0) >= POSITIONAL_LIMITS[pos]:
            suggest.value = f"<b>Cannot pick {pname}:</b> {pos} max reached."; return
        if not feasible_after_pick(your_counts, pos, picks_left, pick_index):
            suggest.value = f"<b>Warning:</b> Picking {pname} may block the 2RB/2WR minimum."

    add_pick_to_state(pick, is_you=is_you)

    if is_you:
        if pos == 'RB': add_handcuff_reminder_if_applicable(pick['Player'])
        remove_handcuff_reminder_if_cuff(pick['Player'])

    pick_index += 1

# ---------------------------
# UNDO
# ---------------------------

def undo_last_pick(_=None):
    global pick_index, drafted_players, your_counts, your_roster, team_counts, pick_history
    if pick_index <= 0 or not pick_history:
        suggest.value = "<i>Nothing to undo.</i>"; return

    last = pick_history.pop()
    pick_index -= 1

    if drafted_players and drafted_players[-1] == last['player']:
        drafted_players.pop()
    else:
        try: drafted_players.remove(last['player'])
        except ValueError: pass

    if last['team'] in team_counts:
        team_counts[last['team']][last['position']] = max(0, team_counts[last['team']].get(last['position'], 0) - 1)

    if last['is_you']:
        your_counts[last['position']] = max(0, your_counts.get(last['position'], 0) - 1)
        for i in range(len(your_roster)-1, -1, -1):
            if your_roster[i]['Player'] == last['player'] and your_roster[i]['Position'] == last['position']:
                your_roster.pop(i); break

    rebuild_handcuff_state_from_roster()
    suggest.value = f"<b>Undid:</b> {last['player']} (Team {last['team']})"
    refresh_ui()

# ---------------------------
# UI WIDGETS
# ---------------------------

title   = HTML("<h3 style='margin:0;'>Live Draft Assistant</h3>")
you_box = IntText(value=YOUR_TEAM_DEFAULT, description='Your Team:', layout=Layout(width='200px'))
status  = HTML("<div style='font-size:12px;'></div>")
suggest = HTML("<div style='font-size:12px;'></div>")

undo_btn = Button(description='Undo Last Pick', tooltip='Revert the last pick you added', layout=Layout(width='140px', height='30px'))
undo_btn.on_click(undo_last_pick)

position_filter = ToggleButtons(options=['All','QB','RB','WR','TE','DST','K'], value='All', description='Filter:')
search   = Text(value="", description='Search:', layout=Layout(width='360px'))
page     = IntSlider(value=1, min=1, max=1, step=1)
per_page = IntSlider(value=10, min=10, max=100, step=2, description='Per page:', layout=Layout(width='320px'))

rows_box = VBox(layout=Layout(border='1px solid #eee', padding='4px', overflow='visible'))
footer_out = Output(layout=Layout(border='1px dashed #ddd', padding='6px'))

prev_btn   = Button(description='‹ Prev', layout=Layout(width='90px', height='30px'))
page_label = HTML("<b>Page 1/1</b>")
next_btn   = Button(description='Next ›', layout=Layout(width='90px', height='30px'))

# --- Roster panel ---
roster_title = HTML("<h4 style='margin:6px 0;'>My Roster</h4>")
roster_out = Output(layout=Layout(border='1px solid #eee', padding='6px'))
needs_out  = Output(layout=Layout(border='1px dashed #ddd', padding='6px', margin='6px 0 0 0'))
handcuff_box = HTML("")

# --- Forecast (Next two teams before you) ---
forecast_title = HTML("<h4 style='margin:6px 0;'>Next two teams before you</h4>")
forecast_html  = HTML(value="")
forecast_panel = VBox([forecast_title, forecast_html],
                      layout=Layout(margin='6px 0 0 0'))

def render_roster():
    with roster_out:
        clear_output(wait=True)
        if not your_roster:
            display(pd.DataFrame(columns=['Pick','Name','Pos','Pts']))
        else:
            df = pd.DataFrame(your_roster)[['Player','Position','Projected_Points']]
            df.insert(0, 'Pick', range(1, len(df) + 1))
            df = df.rename(columns={'Player':'Name','Position':'Pos','Projected_Points':'Pts'})

            # Use HTML to render badge text (pandas styler escapes by default)
            html_rows = []
            for _, r in df.iterrows():
                html_rows.append(
                    f"<div style='display:flex;gap:12px;padding:2px 0;'>"
                    f"<div style='width:36px;text-align:right;'>{int(r['Pick'])}.</div>"
                    f"<div style='min-width:240px;'>{html.escape(r['Name'])}</div>"
                    f"<div style='width:36px;'>{r['Pos']}</div>"
                    f"<div style='width:64px;text-align:right;'>{float(r['Pts']):.1f}</div>"
                    f"</div>"
                )
            display(HTML("<div>" + "".join(html_rows) + "</div>"))

    with needs_out:
        clear_output(wait=True)
        counts = {pos: your_counts.get(pos, 0) for pos in POSITIONAL_LIMITS}
        print("Current counts:", counts)

def render_forecast_panel():
    your_team = int(you_box.value)
    forecast_panel.layout.display = 'none'
    forecast_html.value = ""
    if on_the_clock() != your_team: return
    if current_round(pick_index) % 2 != 0: return
    if pick_index == 0: return
    prev_team = draft_order[pick_index - 1]
    if prev_team != your_team + 1: return

    idxs = intervening_pick_indices(your_team)
    intervening_teams = []
    for i in idxs:
        t = draft_order[i]
        if t not in intervening_teams:
            intervening_teams.append(t)
    if len(intervening_teams) != 2: return

    teams = list(reversed(intervening_teams))
    lines = []
    for t in teams:
        t_idx = max(i for i in idxs if draft_order[i] == t)
        probs = per_team_pos_probs(t, t_idx)
        if not probs: continue
        top = sorted(probs.items(), key=lambda kv: kv[1], reverse=True)[:2]
        parts = [f"{pos} {int(round(prob * 100))}%" for pos, prob in top]
        lines.append(f"<div style='padding:4px 8px;border-bottom:1px solid #eee;'>Team {t}: <b>{' / '.join(parts)}</b></div>")

    if not lines: return
    forecast_html.value = (
        "<div style='border:1px solid #eee;padding:6px;border-radius:8px;background:#f8fafc;'>"
        + "".join(lines) + "</div>"
    )
    forecast_panel.layout.display = ''

def update_status_and_suggest():
    your_team = int(you_box.value)
    who = on_the_clock()
    rp = round_and_pick(pick_index)
    if who == -1:
        status.value = "<b>Draft complete.</b>"
        suggest.value = ""
        return
    mine = " (YOU)" if who == your_team else ""
    status.value = f"<b>On the Clock:</b> Team {who}{mine} &nbsp;&nbsp; <b>{rp}</b>"

    if who == your_team:
        topk = suggest_top_k(SUGGEST_TOP_K, your_team)
        topk = filter_suggestions_by_caps(topk, your_counts)
        if not topk:
            suggest.value = "<b>Suggestions:</b> none (constraints/max reached?)"
        else:
            lines = []
            for i, c in enumerate(topk):
                adp_txt = (f"{int(c.get('ADP_disp'))}" if pd.notna(c.get('ADP_disp')) else
                           (f"{int(c['ADP'])}" if pd.notna(c['ADP']) else "–"))
                t_pct = int(round(100 * c.get('Threat', 0.0)))
                name_html = c.get('NameHTML', html.escape(c['Player']))
                lines.append(f"{i+1}) {name_html} ({c['Position']}) — {c['Projected_Points']:.1f} pts | ADP {adp_txt} | T {t_pct}%")
            suggest.value = "<b>Suggestions:</b><br>" + "<br>".join(lines)
    else:
        suggest.value = "<i>Waiting on other team pick…</i>"

def make_cell(idx: int, pname: str, display_html: str, pos: str, pts: float, adp_disp):
    adp_txt = f"{int(adp_disp)}" if pd.notna(adp_disp) else "–"
    info_html = (f"<div style='font-size:14px; line-height:1.35;'>"
                 f"{idx}. <b>{display_html}</b><br>"
                 f"<span style='color:#333;'>{pos} • {pts:.1f} pts | ADP {adp_txt}</span>"
                 f"</div>")
    info = HTML(info_html)

    btn_other = Button(description='Drafted (Other)', layout=Layout(width='140px', height='32px'))
    btn_me    = Button(description='My Pick', button_style='success', layout=Layout(width='120px', height='32px'))

    def on_other(_):
        apply_pick_by_name(pname, is_you=False); refresh_ui()
    def on_me(_):
        if on_the_clock() != int(you_box.value):
            suggest.value = "<b>Not your turn yet.</b>"; return
        apply_pick_by_name(pname, is_you=True); refresh_ui()

    btn_other.on_click(on_other); btn_me.on_click(on_me)

    return HBox([info, HBox([btn_other, btn_me], layout=Layout(margin='0 0 0 auto'))],
                layout=Layout(justify_content='space-between', align_items='center',
                              padding='6px', overflow='visible', max_height=None))

def render_rows():
    av = current_available(position_filter.value, search.value)
    total = len(av)
    per_col = max(1, int(per_page.value // 2))
    page_block = per_col * 2
    max_pages = max(1, (total + page_block - 1) // page_block)
    if page.value > max_pages: page.value = max_pages
    page.max = max_pages

    start = (page.value - 1) * page_block
    end   = start + page_block

    # include ADP_RT for display
    cols = ['Player','NameHTML','Position','Projected_Points','ADP','ADP_RT']
    cols = [c for c in cols if c in av.columns]
    show  = av.iloc[start:end][cols]

    left_df, right_df = show.iloc[:per_col], show.iloc[per_col:]

    left_cells  = []
    for i, (_, r) in enumerate(left_df.iterrows()):
        left_cells.append(
            make_cell(start + i + 1,
                      str(r['Player']),               # real name for picking
                      r.get('NameHTML', str(r['Player'])),  # HTML with badge
                      r['Position'],
                      float(r['Projected_Points']),
                      r.get('ADP_RT', r.get('ADP')))
        )

    right_cells = []
    for i, (_, r) in enumerate(right_df.iterrows()):
        right_cells.append(
            make_cell(start + per_col + i + 1,
                      str(r['Player']),
                      r.get('NameHTML', str(r['Player'])),
                      r['Position'],
                      float(r['Projected_Points']),
                      r.get('ADP_RT', r.get('ADP')))
        )

    rows_box.children = [HBox([VBox(left_cells,  layout=Layout(flex='1', overflow='visible')),
                               VBox(right_cells, layout=Layout(flex='1', overflow='visible'))],
                              layout=Layout(width='100%'))]

    page_label.value  = f"<b>Page {page.value}/{max_pages}</b>"
    prev_btn.disabled = (page.value <= 1)
    next_btn.disabled = (page.value >= max_pages)

    with footer_out:
        clear_output(wait=True)
        print(f"Showing {len(show)} of {total} available • {page_block} per page")

# ---------------------------
# DRAFT BOARD (by round)
# ---------------------------

def _short_name(full: str) -> str:
    parts = str(full).split()
    name = parts[-1] if parts else str(full)
    return name.replace("’", "'")

_pos_map = dict(zip(players['Player'], players['Position']))

def _player_label(p: str) -> str:
    pos = _pos_map.get(p, "")
    return f"{p} ({pos})" if pos else str(p)

def current_board_df() -> pd.DataFrame:
    cols = [f"P{t}" for t in range(1, TEAMS+1)]
    df = pd.DataFrame("",
                      index=[f"Round {r}" for r in range(1, ROUNDS+1)],
                      columns=cols)
    for i in range(min(pick_index, len(draft_order))):
        rnd = i // TEAMS
        team = draft_order[i]
        if rnd >= ROUNDS: break
        if i < len(drafted_players):
            player = drafted_players[i]
            pos = _pos_map.get(player, "")
            label = f"{player} ({pos})" if pos else f"{player}"
            df.iat[rnd, team-1] = label
    return df

board_title = HTML("<h4 style='margin:6px 0;'>Draft Board (by round)</h4>")
board_html  = HTML(value="")
board_container = VBox(
    [board_html],
    layout=Layout(
        border='1px solid #eee',
        padding='6px',
        max_height='260px',
        width='2230px',
        overflow='auto'
    )
)

def render_board():
    df = current_board_df()
    styled = (
        df.style
          .set_properties(**{"font-size": "12px","white-space": "nowrap","padding": "2px 6px"})
          .set_table_styles([{"selector": "th","props": [("font-size","12px"),("min-width","190px")]}])
    )
    board_html.value = styled.to_html()

# ---------------------------
# REFRESH / WIRING / LAYOUT
# ---------------------------

def refresh_ui(*_):
    render_rows(); update_status_and_suggest(); render_roster(); render_board()
    render_handcuff_panel(); render_forecast_panel()

def _go_prev(_):
    if page.value > 1: page.value -= 1; refresh_ui()

def _go_next(_):
    if page.value < page.max: page.value += 1; refresh_ui()

prev_btn.on_click(_go_prev); next_btn.on_click(_go_next)

position_filter.observe(lambda ch: ch['name'] == 'value' and refresh_ui(), names='value')
search.observe(lambda ch: ch['name'] == 'value' and refresh_ui(), names='value')
per_page.observe(lambda ch: ch['name'] == 'value' and refresh_ui(), names='value')
you_box.observe(lambda ch: ch['name'] == 'value' and refresh_ui(), names='value')

header      = VBox([title, HBox([status, undo_btn], layout=Layout(gap='12px')), suggest], layout=Layout(margin='0 0 6px 0'))
board_panel = VBox([board_title, board_container], layout=Layout(margin='8px 0 0 0'))
controls    = HBox([position_filter, search, you_box], layout=Layout(justify_content='space-between', align_items='center'))
roster_panel= VBox([roster_title, roster_out, needs_out, handcuff_box], layout=Layout(margin='8px 0 0 0'))

app = VBox([
    header,
    forecast_panel,
    board_panel,
    controls,
    rows_box,
    footer_out,
    HBox([prev_btn, page_label, next_btn], layout=Layout(justify_content='center', align_items='center', gap='10px')),
    roster_panel
])

try: app.add_class("draft-app")
except: pass
for box in (rows_box, footer_out, roster_out, needs_out, board_container):
    try: box.add_class("card")
    except: pass

refresh_ui()
display(app)
